In [1]:
import os 
import re
import glob 
from tqdm import tqdm
from itertools import repeat
from collections import Counter
import pandas as pd
import numpy as np
from numpy import array
from IPython.display import display
pd.set_option('display.max_columns', None)
import datetime
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold, KFold ,GridSearchCV ,cross_validate
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier,IsolationForest,StackingClassifier,HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score,balanced_accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,fbeta_score,roc_auc_score,classification_report,make_scorer,balanced_accuracy_score
from mlxtend.classifier import StackingClassifier,StackingCVClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler,Binarizer
from imblearn.over_sampling import SMOTE,ADASYN,BorderlineSMOTE,SVMSMOTE
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import RandomUnderSampler,NearMiss,TomekLinks,CondensedNearestNeighbour,EditedNearestNeighbours
import warnings 
warnings.filterwarnings(action='ignore')

#### 1.) 데이터 기초 전처리 함수 

In [3]:
def load_production(i):
    if i < 39:
        #### 샘플(배치) 데이터 
        if i == 8:
            ### N236072024 배치 생산 데이터 --> 카운트 중간지점부터 재개 [ 이전 생산량 오류로 판명 --> 삭제 ]
            df_sample = pd.read_csv(df_list_2[i])
            df_sample = df_sample[df_sample['일시'] >= '2023-06-07 15:43:16'].reset_index(drop=True)
            df_sample['일시'] = pd.to_datetime(df_sample['일시'])
            df_sample['BatchName'] = df_list_2[i][-14:-4]     
        elif i == 19:
            df_sample = pd.read_csv(df_list_2[i])
            #print('20번 배치  2023-06-15 16:28:13 ~ 2023-06-15 16:28:55 ==> 2001 다음 2080')
            df_sample = df_sample[df_sample['투입수량'] != 2001].reset_index(drop=True)
            df_sample['일시'] = pd.to_datetime(df_sample['일시'])
            df_sample['BatchName'] = df_list_2[i][-14:-4]          
        elif i == 24:
            df_sample = pd.read_csv(df_list_2[i])
            #print('25번 배치  2023-06-16 23:55:10 ~ 2023-06-16 23:57:47 ==> 17859 다음 18145')
            df_sample = df_sample[df_sample['투입수량'] != 17859].reset_index(drop=True)
            df_sample['일시'] = pd.to_datetime(df_sample['일시'])
            df_sample['BatchName'] = df_list_2[i][-14:-4]        
        elif i ==29:
            df_sample = pd.read_csv(df_list_2[i])
            #print('30번 배치  2023-06-17 23:57:24 ~ 2023-06-18 00:00:01 ==> 25197 다음 25483')
            df_sample = df_sample[df_sample['투입수량'] != 25197].reset_index(drop=True)
            df_sample['일시'] = pd.to_datetime(df_sample['일시'])
            df_sample['BatchName'] = df_list_2[i][-14:-4]           
        elif i == 21:
            #### 샘플데이터 22번 --> 인코딩 cp949 별도 & 초단위 시간 별도 적용 필요
            df_sample = pd.read_csv(df_list_2[i],encoding='cp949')
            df_sample['일시'] = pd.to_datetime(df_sample['일시'])
            df_sample['일시'] += pd.TimedeltaIndex(df_sample.groupby('일시').cumcount(), unit='s')
            df_sample['BatchName'] = df_list_2[i][-14:-4]
        else:
            ### 나름 정상 생산 
            df_sample = pd.read_csv(df_list_2[i])
            df_sample['일시'] = pd.to_datetime(df_sample['일시'])
            df_sample['BatchName'] = df_list_2[i][-14:-4]
    else:
        #### 샘플데이터 40번 --> 인코딩 cp949 & 초단위 시간 별도 적용 필요  
        df_sample = pd.read_csv(df_list_2[i],encoding='cp949')
        df_sample = df_sample.rename(columns={'?쇱떆':'일시'})
        df_sample['일시'] = pd.to_datetime(df_sample['일시'])
        df_sample['일시'] += pd.TimedeltaIndex(df_sample.groupby('일시').cumcount(), unit='s')
        df_sample = df_sample[df_sample['일시'] != '2023-06-28 02:35:16'].reset_index(drop=True)
        df_sample['BatchName'] = df_list_2[i][-14:-4]     
    #### Column & 데이터 정보 필터링
    df = df_sample
    df_inputcols = df.drop(['일시','투입수량','양품수량','불량수량','BatchName'],axis=1).head(1)
    df_tagnames = df_inputcols.T.reset_index()[['index']].rename(columns={'index':'태그명'})
    df_tagnames_list = df_tagnames['태그명'].tolist()
    
    #### 태그정보 통합
    #df_info = pd.read_excel('C:/Users/sangb/OneDrive/바탕 화면/LS엠트론/사출데이터/사출데이터/INM005 tag list.xlsx')
    df_info = pd.read_excel('C:/Users/user/Desktop/데이터분석 프로젝트/LS엠트론/사출데이터/INM005 tag list.xlsx')
    df_info = df_info[df_info['태그명'].isin(df_tagnames_list) == True]
    df_info = df_info.dropna(axis=1).reset_index(drop=True)
    #df_info_2 = pd.read_csv('C:/Users/sangb/OneDrive/바탕 화면/LS엠트론/사출데이터/사출데이터/태그 정리_v2.csv',encoding='cp949')
    df_info_2 =  pd.read_csv('C:/Users/user/Desktop/데이터분석 프로젝트/LS엠트론/사출데이터/태그 정리_v2.csv',encoding='cp949')
    df_taglist = pd.merge(df_info,df_info_2[['태그설명','구분']],on='태그설명',how='inner').drop_duplicates().reset_index(drop=True)
    
    #### 태그설명 완료 데이터 
    df_tag = df_sample.drop(['BatchName','일시','투입수량','양품수량','불량수량'],axis=1)
    df_nontag = df_sample[['BatchName','일시','투입수량','양품수량','불량수량']]
    df_tag.columns = df_taglist['태그설명'].tolist()
    df_sample['사출기'] = 'INM005 05호기(수평)'
    df_added = pd.concat([df_sample['사출기'],df_nontag,df_tag],axis=1)
    
    #### 기존 배치 단위 데이터에서 누적투입/양품/불량수량 동시 = 0 제외 ==> 생산 대기중 가정 
    df_existing_counts = df_added[(df_added['투입수량'] !=0) & (df_added['양품수량'] !=0)]
    
    #### 배치 데이터 인덱스 부여 
    df_production = df_existing_counts
    df_production = df_production.reset_index(drop=True)
    df_production['sampledata_index'] = i+1
    df_production = df_production.rename(columns={'일시':'생산시간'})
    return df_production

#### 2.) 생산데이터 표준화 함수 

In [4]:
def cycletime_indexing(i):
    start = 1
    integer = 0
    decimal = 0
    group_number = 0
    lag = 2 # lag custom 지정 가능 
    result = []
    result_group_number = []
    total_df = pd.DataFrame()
    df = load_production(i) # cycletime 3.6 ~ 5.9초만 활용 
    df = df[(df['사출기 : 실적표시:사이클시간_Actual_v.:Cycle_time'] >=3)&(df['사출기 : 실적표시:사이클시간_Actual_v.:Cycle_time'] <6)]
    df = df.reset_index(drop=True)
    # 수량 한정 시차 이동 
    count_shift = df[['투입수량','양품수량','불량수량']].shift(lag) # lag 0 --> 시차이동 X 
    df[['투입수량','양품수량','불량수량']] = count_shift
    df = df.dropna().reset_index(drop=True)
    for n, i in enumerate(range(len(df))): 
        if start == 1:   
            integer, decimal_point = str(df.iloc[i]['사출기 : 실적표시:사이클시간_Actual_v.:Cycle_time']).split('.')
            integer, decimal_point = int(integer), float(decimal_point)
            if decimal >= 1:
                decimal -= 1
                integer += 1
            if n == 0:
                result_group_number.append(group_number)
                result.append(df.iloc[i])
                start += 1
                decimal = decimal+ decimal_point*0.1
            else:
                result_group_number.append(group_number)
                group_number+=1
                start += 1
                result_group_number.append(group_number)
                result.append(df.iloc[i])
                result.append(df.iloc[i])
                decimal = decimal+ decimal_point*0.1
        else:
            result_group_number.append(group_number)
            result.append(df.iloc[i])
            start += 1
            if start < integer: continue
            else: start -= integer
    df1 = pd.DataFrame(result, columns = df.columns)
    df1['group'] = result_group_number
    df1 = df1[df1.group != group_number]
    total_df = pd.concat([total_df,df1])
    return total_df

#### 3.) 데이터 추가 전처리 함수1

In [5]:
def preprocess_production():
    #### 1.) 데이터 통합 
    dfs = []
    for i in data_list:
        total_df = cycletime_indexing(i)
        print('Data No.',i+1,' ==> Add CycleTime_Index',' ',len(total_df))
        dfs.append(total_df) 
    df_concat = pd.concat(dfs).reset_index(drop=True)
    info_cols = df_concat.columns[:6].tolist() + df_concat.columns[-2:].tolist()
    input_cols = df_concat.columns[6:-2].tolist()
    production = df_concat[info_cols + input_cols]

    #### 2.) Setting & Actual 변수 분리 
    df_inputcols = production[input_cols].head(1)
    df_tagnames = df_inputcols.T.reset_index()[['index']].rename(columns={'index':'태그명'})
    df_tagnames_list = df_tagnames['태그명'].tolist()
    df_info = pd.read_excel('C:/Users/user/Desktop/데이터분석 프로젝트/LS엠트론/사출데이터/INM005 tag list.xlsx')
    df_info = df_info[df_info['태그설명'].isin(df_tagnames_list) == True]
    df_info = df_info.dropna(axis=1).reset_index(drop=True)
    df_info_2 = pd.read_csv('C:/Users/user/Desktop/데이터분석 프로젝트/LS엠트론/사출데이터/태그 정리_v2.csv',encoding='cp949')
    df_taglist = pd.merge(df_info,df_info_2[['태그설명','구분']],on='태그설명',how='inner').drop_duplicates().reset_index(drop=True)
    df_settings = []
    for j in range(0,len(df_taglist)):
        if 'sv' in df_taglist['구분'].iloc[j]:
            df_settings.append('True')
        elif 'SV' in df_taglist['구분'].iloc[j]:
            df_settings.append('True')
        else:
            df_settings.append('False')
    df_taglist['is_setting'] = pd.DataFrame(df_settings) 
    tag_divided_info = df_taglist[df_taglist['is_setting'] == 'False'][['태그설명','구분']]
    setting_cols = df_taglist[df_taglist['is_setting'] == 'True']['태그설명'].tolist()
    actual_cols = df_taglist[df_taglist['is_setting'] == 'False']['태그설명'].tolist()

    #### 3.) Setting값  제거  
    production = production[info_cols + actual_cols]

    #### 4.) 통합 데이터 기준 총변동 1회인 Actual값 사전 제거 [ 모든 데이터에서 같은 조건의 수치를 가진 변수 ]
    col_count = pd.DataFrame(production[actual_cols].nunique()).sort_values(by=0,ascending=True).reset_index().rename(columns={'index':'태그설명',0:'Count'})
    production = production[production.columns[:8].tolist() + col_count[col_count['Count'] >1]['태그설명'].tolist()]
    print('\n')
    print('Setting Values ==>',len(setting_cols))
    print('Original Actual Values =',len(actual_cols),' ','==>  Cut Actual Values=',len(production.columns[8:].tolist()))
    return production

#### 4.) 데이터 추가 전처리 함수2

In [6]:
def production_filter(i):
    production_connect = production
    production_connect = production_connect.rename(columns={'group':'cycletime_index'})
    
    #### 생산데이터 선택 ==> Actual 입력변수 & CycleTime_Index 정의 
    sampledata = production_connect[production_connect['sampledata_index']== i+1]
    actual_cols_list = production_connect.columns[8:].tolist()
    cycletime_index_list = sampledata['cycletime_index'].unique().tolist()
    
    #### 1.) 투입 생산수량이 증가하지 않는 부분행 데이터 전부 제거 
    quantity_increase = []
    for k in cycletime_index_list:
        sampledata_cycleindex =  sampledata[sampledata['cycletime_index']==k]
        cycletime_index = sampledata_cycleindex['cycletime_index'].unique()[0]
        if sampledata_cycleindex['투입수량'].nunique() !=1:
            quantity_increase.append(cycletime_index)
    production_cut = sampledata[sampledata['cycletime_index'].isin(quantity_increase) == True]
    production_cut = production_cut.reset_index(drop=True)
    print('투입수량 증가하는 CycleTime_Index 식별 ==> Data No.',i+1,'',production_cut['BatchName'].unique()[0],'',len(production_cut),'행')
    return production_cut

def preprocess_filter():
    df_filtered = []
    for i in data_list:
        production_cut = production_filter(i)
        if len(production_cut) !=0:    
            df_filtered.append(production_cut)
    production_cut_1 = pd.concat(df_filtered).reset_index(drop=True)
    
    #### 총변동 1회인 Actual값 사전 제거 [ 모든 데이터에서 같은 조건의 수치를 가진 변수 ]
    actual_cols = production_cut_1.columns[8:].tolist()
    col_count = pd.DataFrame(production_cut_1[actual_cols].nunique()).sort_values(by=0,ascending=True).reset_index().rename(columns={'index':'태그설명',0:'Count'})
    production_filtered = production_cut_1[production_cut_1.columns[:8].tolist() + col_count[col_count['Count'] >1]['태그설명'].tolist()]
    return production_filtered

#### 5.) 데이터 추가 전처리 함수3

In [7]:
def data_selection(i):
    select_data = production_filtered[production_filtered['sampledata_index'] == i+1]
    return select_data

def change_detection(j):
    select_data = data_selection(i)
    input_cols = select_data.columns[8:].tolist()
    split_cycletime = select_data[select_data['cycletime_index']==j].reset_index(drop=True)
    split_cycletime_inputs = split_cycletime[input_cols]
    check_all_unique = split_cycletime_inputs.loc[:,split_cycletime_inputs.apply(pd.Series.nunique) != 1]
    change_detect = len(check_all_unique.columns.tolist())
    if change_detect >0:
        split_cycletime['Change_Detection'] = 'True'
    else:
        split_cycletime['Change_Detection'] = 'False'     
    return split_cycletime

def add_change_detection(i):
    select_data = data_selection(i)
    cycletime_index = select_data['cycletime_index'].unique().tolist()
    dfs = []
    for j in cycletime_index:
        split_cycletime = change_detection(j)
        dfs.append(split_cycletime)
    production_sample = pd.concat(dfs).reset_index(drop=True)
    return production_sample

#### 6.) 생산데이터 최종  분석용 집계함수 

In [39]:
def groupby(i):
    select_data = production_added[production_added['sampledata_index']==i+1]
    input_counts = ['투입수량','양품수량','불량수량']
    input_cols = select_data.columns[9:-5].tolist()
    info_cols = ['BatchName','sampledata_index']
    time_col = ['생산시간']

    #### 정보 나열 
    df_info = select_data.groupby('cycletime_index')[info_cols].first()

    #### CyleTime_Index당 수량 증가량 
    count_first = select_data.groupby('cycletime_index')[input_counts].first()
    count_last = select_data.groupby('cycletime_index')[input_counts].last()
    inputcount_increments = count_last - count_first
    
    #### CycleTime_Index당 누적수량 
    #count_max = select_data.groupby('cycletime_index')[input_counts].max()
    #count_max.columns = count_max.columns + "(누적)"

    #### CycleTime_Index당 시작 ~ 종료 시점 
    cycle_start = select_data.groupby('cycletime_index')[time_col].first().rename(columns={'생산시간':'생산시작'})
    cycle_last = select_data.groupby('cycletime_index')[time_col].last().rename(columns={'생산시간':'생산종료'})
    df_time = pd.concat([cycle_start,cycle_last],axis=1)

    #### CycleTime_Index당 통계량 
    #input_min = select_data.groupby('cycletime_index')[input_cols].min()
    #input_min.columns = input_min.columns + "_Min"
    
    #input_max = select_data.groupby('cycletime_index')[input_cols].max()
    #input_max.columns = input_max.columns + "_Max"
    
    input_mean = select_data.groupby('cycletime_index')[input_cols].mean()
    input_mean.columns = input_mean.columns + "_Mean"

    #### CycleTime_Index당 Change Detection 
    change_detection = select_data.groupby('cycletime_index')['Change_Detection'].agg(lambda x:x.value_counts().index[0])
    df_groupby = pd.concat([df_info,df_time,inputcount_increments,input_mean,change_detection],axis=1)
    
    #### CycleTime_Index당 집계된 투입수량 ==8 [ 우선 적용 ]
    df_groupby = df_groupby[df_groupby['투입수량'] ==8].reset_index(drop=True)
    return df_groupby

#### 7.) 생산데이터 업로드
- 배치 오류/ 투입수량 Count 수치 이상 제거 
- Target 시차 이동 설정 

In [9]:
#### 활용할 배치 데이터 
data_list = [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 36, 37, 38, 39]
#### 데이터 저장 경로 설정 
path = 'C:/Users/user/Desktop/데이터분석 프로젝트/LS엠트론/사출데이터/CSV 사출데이터'
df_list_2 = glob.glob(path +'\*.csv') 
print('생산 샘플 csv 개수 -->',len(df_list_2))

생산 샘플 csv 개수 --> 40


In [10]:
# 시차 이동 예시 
lag = 2 

df = load_production(0) # cycletime 3.6 ~ 5.9초만 활용 
df = df[(df['사출기 : 실적표시:사이클시간_Actual_v.:Cycle_time'] >=3)&(df['사출기 : 실적표시:사이클시간_Actual_v.:Cycle_time'] <6)]
df = df.reset_index(drop=True)

# 검사수량 한정 시차 이동 
count_shift = df[['투입수량','양품수량','불량수량']].shift(lag) # lag 0 --> 시차이동 X 
df[['투입수량','양품수량','불량수량']] = count_shift
df = df.dropna().reset_index(drop=True)

df[['BatchName','생산시간','투입수량','양품수량','불량수량']]

,BatchName,생산시간,투입수량,양품수량,불량수량
0,N236012024,2023-06-01 19:31:29,1.0,1.0,0.0
1,N236012024,2023-06-01 19:31:30,4.0,4.0,0.0
2,N236012024,2023-06-01 19:31:31,7.0,7.0,0.0
3,N236012024,2023-06-01 19:31:32,7.0,7.0,0.0
4,N236012024,2023-06-01 19:31:33,7.0,7.0,0.0
...,...,...,...,...,...
13467,N236012024,2023-06-01 23:15:56,25807.0,25081.0,726.0
13468,N236012024,2023-06-01 23:15:57,25807.0,25081.0,726.0
13469,N236012024,2023-06-01 23:15:58,25807.0,25081.0,726.0
13470,N236012024,2023-06-01 23:15:59,25807.0,25081.0,726.0


#### 8.) CycleTime 4,5초에 해당하는 데이터 인덱스 부여 
- Setting 값 제거 &  Actual 입력변수 기준 획일값 1차 제거 
- 수량 증가량 0인 Index 부분행 제거 
- Actual 입력변수 기준 획일값 2차 제거 

In [11]:
production = preprocess_production() # Setting값 제거 & 획일값 제거 

Data No. 1  ==> Add CycleTime_Index   16764
Data No. 2  ==> Add CycleTime_Index   17387
Data No. 3  ==> Add CycleTime_Index   19469
Data No. 4  ==> Add CycleTime_Index   17668
Data No. 6  ==> Add CycleTime_Index   16993
Data No. 7  ==> Add CycleTime_Index   17160
Data No. 8  ==> Add CycleTime_Index   22786
Data No. 9  ==> Add CycleTime_Index   17890
Data No. 10  ==> Add CycleTime_Index   17392
Data No. 11  ==> Add CycleTime_Index   20087
Data No. 12  ==> Add CycleTime_Index   18068
Data No. 13  ==> Add CycleTime_Index   18830
Data No. 14  ==> Add CycleTime_Index   19534
Data No. 16  ==> Add CycleTime_Index   17447
Data No. 17  ==> Add CycleTime_Index   18911
Data No. 18  ==> Add CycleTime_Index   17724
Data No. 20  ==> Add CycleTime_Index   17804
Data No. 21  ==> Add CycleTime_Index   25921
Data No. 22  ==> Add CycleTime_Index   26532
Data No. 23  ==> Add CycleTime_Index   19877
Data No. 25  ==> Add CycleTime_Index   18627
Data No. 26  ==> Add CycleTime_Index   19995
Data No. 27  ==> A

In [12]:
production_filtered = preprocess_filter() # 수량 증가량 0인 인덱스에 해당하는 부분행 제거 & 획일값 제거 

투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 1  N236012024  16433 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 2  N236022007  17107 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 3  N236022013  17397 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 4  N236062019  16962 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 6  N236072005  16609 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 7  N236072007  16772 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 8  N236072012  16681 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 9  N236072024  17335 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 10  N236132019  17274 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 11  N236132025  17341 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 12  N236132031  17253 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 13  N236142001  17519 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 14  N236142015  15503 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 16  N236142025  17345 행
투입수량 증가하는 CycleTime_Index 식별 ==> Data No. 17  N236142033  17178 행
투입수량 증가하는 CycleTim

In [13]:
production_filtered.shape

(583000, 78)

In [14]:
#### 남은 획일값 입력변수 확인 
production_filtered[production_filtered.columns[8:].tolist()].loc[:,production_filtered[production_filtered.columns[8:].tolist()].nunique()==1].shape

(583000, 0)

#### 9.) Index별 Groupby 
- 각 CycleTime_Index당 파생변수 생성 [ Mean/Change_Detection ] 
- Groupby 이후 각 Index당 수량 증가량 8인 행 선택
- 획일값 3차 탐지 

In [15]:
dfs = []
for i in data_list:
    production_sample = add_change_detection(i)
    print('Data No.',i+1,' ==> Add Change Detection Column')
    dfs.append(production_sample)   
production_added = pd.concat(dfs).reset_index(drop=True)

Data No. 1  ==> Add Change Detection Column
Data No. 2  ==> Add Change Detection Column
Data No. 3  ==> Add Change Detection Column
Data No. 4  ==> Add Change Detection Column
Data No. 6  ==> Add Change Detection Column
Data No. 7  ==> Add Change Detection Column
Data No. 8  ==> Add Change Detection Column
Data No. 9  ==> Add Change Detection Column
Data No. 10  ==> Add Change Detection Column
Data No. 11  ==> Add Change Detection Column
Data No. 12  ==> Add Change Detection Column
Data No. 13  ==> Add Change Detection Column
Data No. 14  ==> Add Change Detection Column
Data No. 16  ==> Add Change Detection Column
Data No. 17  ==> Add Change Detection Column
Data No. 18  ==> Add Change Detection Column
Data No. 20  ==> Add Change Detection Column
Data No. 21  ==> Add Change Detection Column
Data No. 22  ==> Add Change Detection Column
Data No. 23  ==> Add Change Detection Column
Data No. 25  ==> Add Change Detection Column
Data No. 26  ==> Add Change Detection Column
Data No. 27  ==> A

In [42]:
production_added.shape

(583000, 79)

In [43]:
#### 남은 획일값 입력변수 확인 
production_added[production_added.columns[8:].tolist()].loc[:,production_added[production_added.columns[8:].tolist()].nunique()==1].shape

(583000, 0)

In [40]:
dfs = []
for i in data_list:
    df_groupby = groupby(i)
    dfs.append(df_groupby)
    print('Data No.',i+1,' ==> Groupby CycleTime_Index') 
df_groupby_concat = pd.concat(dfs).reset_index(drop=True)
df_groupby_concat = df_groupby_concat.drop(['사출기 : 실적표시:사이클시간_Actual_v.:Cycle_time_Mean'],axis=1)
df_groupby_concat['Change_Detection'] = df_groupby_concat['Change_Detection'].replace({'False':0,'True':1})

Data No. 1  ==> Groupby CycleTime_Index
Data No. 2  ==> Groupby CycleTime_Index
Data No. 3  ==> Groupby CycleTime_Index
Data No. 4  ==> Groupby CycleTime_Index
Data No. 6  ==> Groupby CycleTime_Index
Data No. 7  ==> Groupby CycleTime_Index
Data No. 8  ==> Groupby CycleTime_Index
Data No. 9  ==> Groupby CycleTime_Index
Data No. 10  ==> Groupby CycleTime_Index
Data No. 11  ==> Groupby CycleTime_Index
Data No. 12  ==> Groupby CycleTime_Index
Data No. 13  ==> Groupby CycleTime_Index
Data No. 14  ==> Groupby CycleTime_Index
Data No. 16  ==> Groupby CycleTime_Index
Data No. 17  ==> Groupby CycleTime_Index
Data No. 18  ==> Groupby CycleTime_Index
Data No. 20  ==> Groupby CycleTime_Index
Data No. 21  ==> Groupby CycleTime_Index
Data No. 22  ==> Groupby CycleTime_Index
Data No. 23  ==> Groupby CycleTime_Index
Data No. 25  ==> Groupby CycleTime_Index
Data No. 26  ==> Groupby CycleTime_Index
Data No. 27  ==> Groupby CycleTime_Index
Data No. 28  ==> Groupby CycleTime_Index
Data No. 29  ==> Groupby

In [49]:
df_groupby_concat.to_csv('df_v2_lag2.csv',encoding='cp949',index=False)